### Interpolating CO$_{2}$ wt.% lost at depth 

---

Author: Christopher M. Gonzalez

Contact: christophmgonzalez@gmail.com

Phone: +610426940721

---

Purpose: Calculate the amount of CO$_{2}$ lost at depth using a _look-up_ table approach. Uses only standard libraries of numpy and scipy.interpolate.

Reads in gridded \*.dat information as a function of P(ressure), T(emperature), and CO$_2$/H$_2$O wt.% into an array CO2Array_TPZ. Returns the interpolated CO2 wt.% at depth


Relevant equations:

$\phi_{T,P} = A_{Ma} \times V_{slab} \times sin(\delta) $

Where $\phi$ is the thermal parameter, $A$ - Age of the slab in years, $V_{slab}$ - Velocity of the slab in km yr$^{-1}$, $\delta$ - slab dip in degrees.

Slab top temperatures are derived from Van Keken et. al (2011) from section 3.1 at the following reference:

```Van Keken, P. E., Hacker, B. R., Syracuse, E. M., & Abers, G. A. (2011). Subduction factory: 4. Depth-dependent flux of H2O from subducting slabs worldwide. Journal of Geophysical Research: Solid Earth, 116(1). https://doi.org/10.1029/2010JB007922```

Their logic is that at depth, the temperature of the slab can be reduced to a log-linear Temperature relationship as a function of the thermal parameter ($\phi$) for the following lithologies: Sediments, Volcanics, and the temperature at the Oceanic crustal moho (i.e., the base of the intrusives ~7km depth). <br> Since the an approximation for sub-arc depths (ranges from ~70 - 200 km) with the median ~100-120 km depth from continental-onceanic and intraoceanic arc ranges 90-160 without a clear peak in its distribution (e.g., Schmidt and Poli 2013). 

Total CO2 loss can be calculated as follows: 

 CO$_{2}$$_{Loss}$ = CO$_{2}$$_{Final}$ - CO$_{2}$$_{initial}$

Where the integrated loss of CO2 can be calculated as a function of each lithology.

N.B. the following assumptions:
1. CO2 loss is only calculated as a point value difference and not as a path function
2. This is an extremely conservative value, meaning CO2 loss is due strictly to thermal breakdown and not due to infiltration of water from subjacent lithologies. 
3. All CO2 loss in this physically implausible model suggest it is lost in batch process at this depth.
4. Relies on the validity of the thermal parameter and applying some form of uniformitarianism, it is also applicable back in time. 


P, T, [CO2 | H2O] wt.% data is calculated on a regular grid of 313 x 313 nodes using the thermodynamic modelling software: Perple_X (Connolly, 2009; http://www.perplex.ethz.ch/).

Perple_X version information: ```Perple_X version 6.9.0, source updated October 11, 2020```

```
Grid parameters:
T_{0}: 573 Kelvin
P_{0}: 500 bars
Pressure resolution: 286.85897435897436 bars
Temperature resolution: 3.5256410256410255 Kelvins
```

References: 
---

```
Connolly, J. A. D. (2009). The geodynamic equation of state: What and how. Geochemistry, Geophysics, Geosystems, 10(10). https://doi.org/10.1029/2009GC002540

Schmidt, M. W., & Poli, S. (2014). Devolatilization During Subduction. In H. D. Turekian & K. HollandKarl (Eds.), Treatise on Geochemistry (2nd ed., Vol. 4, pp. 669–701). Elsevier. https://doi.org/10.1016/B978-0-08-095975-7.00321-1

```


In [1]:
import numpy as np
from scipy.interpolate import griddata

In [2]:
# Take in values of slab dip and return phi, the thermal parameter
def calcPhi(slabAge, slabVelocity, slabDip):
    # phiTP = A(ge) * V(elocity) * sin(\delta)
    # Age: Myrs -> yrs
    slabAge*=1e+6
    # Slab velocity: cm/year -> km/yr
    slabVelocity*=1e-5
    # Slab dip calculation, note numpy does things in radians, need to convert to degrees
    slabDip = np.sin(np.deg2rad(slabDip))
    print("Slab age: {} years \n Slab Velocity: {} km yr \n sin(d): {} \n Product: {}".format(slabAge, slabVelocity, slabDip, slabAge*slabVelocity*slabDip))
    # Return the product phiTP
    return slabAge*slabVelocity*slabDip

# Calculate temperature, equations
def calcSlabTemperatures(Phi):
    T_Sediments = 1331. - 58.6*np.log(Phi)
    T_Volcanics = 1303. - 60.23*np.log(Phi)
    T_Moho      = 1622. - 132.5*np.log(Phi)
    
    print("TSed: {} \n TVolc: {} \n TMoho: {}".format(T_Sediments, T_Volcanics, T_Moho))
    
    return T_Sediments, T_Volcanics, T_Moho

def interpolate(P_particle, T_particle, lithology):
    # Interpolation function. Takes in the approximated Depth to pressure conversion and calculated temperature from the slab tops using
    # Van Keken (2011) and the thermal parameter. 
    # Uses scipy.interpolate griddata as the interpolating function as opposed to my home-written method. 
    
    #                                              0   1    2
    # Read in CO2 data generated from Perple_X.  [ T,  P,  CO2]
    if lithology==0:
        CO2Array_TPZ = np.genfromtxt('Sediments/GLOSS_CO2.dat',
                                  skip_header=12, autostrip=True)
    elif lithology==1:
        CO2Array_TPZ = np.genfromtxt('Metabasalts/StaudigelVolcanics_CO2.dat',
                                  skip_header=12, autostrip=True)
    elif lithology==2: 
        CO2Array_TPZ = np.genfromtxt('Intrusives/Intrusives_CO2.dat',
                                  skip_header=12, autostrip=True)
    else: 
        CO2Array_TPZ = np.genfromtxt('Sublithospheric_Oceanic_mantle/LOSIMAG_CO2.dat',
                                  skip_header=12, autostrip=True)
    
    return griddata( np.array( ( CO2Array_TPZ[:, 0], CO2Array_TPZ[:, 1]) ).T , CO2Array_TPZ[:, 2], (T_particle, P_particle), method='nearest' ) 

In [3]:
if __name__ == '__main__':
    # Calculations are for a constant pressure (4.3750 GPa) at 125 km
    # Assumed 10 m/s (gravity) and density of 3500 kg/m3
    PConstant = 125.*10*35
    
    # Calculate PHI using the theraml parameter:
    # Kirby, S. H., W. B. Durham, and L. A. Stern (1991), 
    # Mantle phase changes and deep‐earthquake faulting in subducted lithosphere, 
    # Science, 252, 216–225, doi:10.1126/science.252.5003.216.
    
    Phi = calcPhi(55.9, 6.34, 53.7)
    
    # Calculate slab top temperatures from Section 3.1: 
    # Van Keken, P. E., Hacker, B. R., Syracuse, E. M., & Abers, G. A. (2011). 
    # Subduction factory: 4. Depth-dependent flux of H2O from subducting slabs worldwide. 
    # Journal of Geophysical Research: Solid Earth, 116(1). https://doi.org/10.1029/2010JB007922
    
    # Returns Temperatures in Celcius 
    T_Sediments, T_Volcanics, T_Moho = calcSlabTemperatures(Phi)
    
    
    
    # Calculate CO2 wt% at 125 km, slab top temperature derived from linear-log regression of Van Keken:
    # Perple_X is calculated in Kelvins
    CO2Sediments  = interpolate(PConstant, T_Sediments + 273.15, 0)
    CO2Volcanics  = interpolate(PConstant, T_Volcanics + 273.15, 1)
    CO2Intrusives = interpolate(PConstant, T_Moho + 273.15, 2)
    
    print("Sediments: {} \n Volcanics: {} \n Intrusives {}".format(CO2Sediments, CO2Volcanics, CO2Intrusives))

Slab age: 55900000.0 years 
 Slab Velocity: 6.340000000000001e-05 km yr 
 sin(d): 0.8059282822485159 
 Product: 2856.2581879856757
TSed: 864.704111530107 
 TVolc: 823.7337651443403 
 TMoho: 567.6620269238765
Sediments: 0.999875 
 Volcanics: 2.54954 
 Intrusives 0.0197072


In [4]:
Phi

2856.2581879856757

In [4]:
interpolate(PConstant, T_Moho+273.14, 3)

0.00990845